In [59]:
from threading import Thread, Lock
from time import sleep
from contextlib import contextmanager

class BinaryLock:
    def __init__(self):
        self.write_lock_obj = Lock()
        self.read_lock_obj = False
    
    @contextmanager
    def read_lock(self):
        print("\nRead_lock called:")
        try :
            if not self.write_lock_obj.locked():
                print("Acquire read_lock")
                self.read_lock_obj = True
            else:
                print("Wait for write_lock")
            yield
        finally:
            print("Release read_lock")
            self.read_lock_obj = False
    
    @contextmanager
    def write_lock(self):
        print("\nWrite_lock called:")
        try :
            if not self.read_lock_obj:
                print("Acquire write_lock")
                self.write_lock_obj.acquire()
            else:
                print("Wait for read_lock")
            yield
        finally:
            if not self.read_lock_obj:
                print("Release write_lock")
                self.write_lock_obj.release()

In [60]:
x = 0
def read(lock):
    global x
    with lock.read_lock():
        print("Shared resource value: {}".format(x))
        sleep(2)

def write(num, lock):
    global x
    with lock.write_lock():
        local = x
        local += num
        x = local
        
lock = BinaryLock()

t1 = Thread(target=read, args=(lock,))
t2 = Thread(target=write, args=(10, lock))
t3 = Thread(target=read, args=(lock,))
t4 = Thread(target=write, args=(20, lock))
t5 = Thread(target=write, args=(30, lock))
t6 = Thread(target=read, args=(lock,))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()

print(f'\nThe final counter is {x}')


Read_lock called:
Acquire read_lock
Shared resource value: 0

Write_lock called:
Wait for read_lock

Read_lock called:
Acquire read_lock
Shared resource value: 10

Write_lock called:
Wait for read_lock

Write_lock called:
Wait for read_lock

Read_lock called:
Acquire read_lock
Shared resource value: 60
Release read_lock
Release read_lock
Release read_lock

The final counter is 60
